In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 128
max_seq_len = 80
batch_size = 64
max_iters = 1000
eval_iters = 100
eval_interval = 500
learning_rate = 3e-4
n_embd = 384
n_layer = 8
n_head  = 8
dropout = 0.2

cuda


In [4]:
# from datasets import load_dataset
# import random

# # Load the dataset with trust_remote_code=True
# dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True, trust_remote_code=True)
# print("Dataset loaded.")

# # Save the entire dataset into a text file
# print("Saving the entire dataset to all_data.txt...")
# with open("all_data.txt", "w", encoding="utf-8") as f:
#     for example in dataset:
#         f.write(example["text"])
#         f.write("\n")

# print("All data saved to all_data.txt.")

# # Read the entire dataset from the saved file
# with open("all_data.txt", "r", encoding="utf-8") as f:
#     all_data = f.read()

# # Split the data into training and validation sets (90/10 split)
# split_index = int(0.9 * len(all_data))
# train_data = all_data[:split_index]
# val_data = all_data[split_index:]

# # Save the training and validation data into separate files
# print("Saving the training and validation data to train.txt and val.txt...")
# with open("train.txt", "w", encoding="utf-8") as f:
#     f.write(train_data)

# with open("val.txt", "w", encoding="utf-8") as f:
#     f.write(val_data)

# print("Training and validation data saved.")

# # # Extract unique characters and compute vocabulary size from the entire dataset
# # print("Extracting unique characters and computing vocabulary size...")
# # chars = sorted(list(set(all_data)))
# # vocab_size = len(chars)
# # print(f"Vocabulary size: {vocab_size}")


In [5]:
# import os

# # Define file paths
# input_file = "all_data.txt"
# train_file = "train.txt"
# val_file = "val.txt"

# # Define the split ratio
# split_ratio = 0.9

# # Get the size of the input file
# file_size = os.path.getsize(input_file)

# # Calculate the split index based on the file size
# split_index = int(file_size * split_ratio)

# # Open the input file and the output files
# with open(input_file, "r", encoding="utf-8") as infile, \
#      open(train_file, "w", encoding="utf-8") as train_out, \
#      open(val_file, "w", encoding="utf-8") as val_out:
    
#     current_pos = 0
#     for line in infile:
#         # Write to the training file until the split index is reached
#         if current_pos < split_index:
#             train_out.write(line)
#         else:
#             val_out.write(line)
        
#         # Update the current position
#         current_pos += len(line.encode('utf-8'))

# print("Training and validation data saved.")


In [ ]:
input_file = "all_data.txt"

# Use a set to track unique characters
unique_chars = set()

# Read the file line by line and add characters to the set
with open(input_file, "r", encoding="utf-8") as infile:
    for line in infile:
        unique_chars.update(line)

# Sort the unique characters and calculate the vocabulary size
chars = sorted(list(unique_chars))
vocab_size = len(chars)

print(f"Vocabulary size: {vocab_size}")

In [ ]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# data = torch.tensor(encode(text), dtype = torch.long)
# print(data[:100])

In [ ]:
def get_random_chunk(split):
    filename = 'train.txt' if split =='train' else 'val.txt'
    with open(filename,'rb') as f:
        with mmap.mmap(f.fileno(), 0, access = mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos = random.randint(0,(file_size) - block_size*batch_size)
            mm.seek(start_pos)
            block = mm.read(batch_size*block_size -1)
            decoded_block = block.decode('utf-8' , errors = 'ignore').replace('\r' , '')
            data = torch.tensor(encode(decoded_block), dtype = torch.long)
    return data
    

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y

x,y = get_batch('train')
    

In [ ]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train' , 'val']:
        losses = torch.zeros(eval_iters)
        for k in range (eval_iters):
            X,Y = get_batch(split)
            logits,loss = model.forward(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    


In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        if self.tril.size(0) < T or self.tril.size(1) < T:
            raise ValueError(f"The tril tensor is too small for sequence length T={T}.")
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

        
    
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.head = nn.ModuleList([Head(head_size) for _ in range (num_heads) ])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.head], dim = -1)
        out = self.dropout(self.proj(out))
        return out
          

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self,x):
         return self.net(x)
        
            
        

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln_1 = nn.LayerNorm(n_embd)
        self.ln_2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln_1(x+y)
        y = self.ffwd(x)
        x = self.ln_2(x+y)
        return x
        
    

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size+1,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self.__init__weights)

    def __init__weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0,  std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0,  std = 0.02)
    
    def forward(self, index, targets=None):
        B, T = index.shape
        if torch.max(index) >= vocab_size:
            raise IndexError(f"Token index out of range. Max index: {torch.max(index)}, Vocab size: {vocab_size}")
        tok_emb = self.token_embedding_table(index)
        if T > block_size:
            raise IndexError(f"Position index out of range. T: {T}, Block size: {block_size}")
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            if index.size(1) == block_size:

            # Process the sequence in chunks of block_size
                (logits, loss) = self.forward(index)
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                index_next = torch.multinomial(probs, num_samples=1)
                index = index_next
            else:
                (logits, loss) = self.forward(index)
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                index_next = torch.multinomial(probs, num_samples=1)
                index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)

# print('Model loading parameters')
# with open('model-01.pkl','rb') as f:
#     model = pickle.load(f)
# print('loaded successfully')
m = model.to(device)
    
    

            
        

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range (max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(losses['train'] ,"   " , losses['val'])
    xb,yb = get_batch('train')
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb' ) as f:
    pickle.dump(model,f)
print("model saved")
    

In [ ]:
context = torch.zeros((1,1) , dtype = torch.long , device = device)
gen_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(gen_chars)